In [1]:
import pyomo.environ as pe
import pyomo.opt as po


## Charter flights

Cristina Hernández, Macarena Vargas, Guillermo Ruiz, Beatriz Jiménez. 3ºIMAT B


Resumen del enunciado: 

Una compañía tiene 5 aviones de cada tipo (A,B,C). Cada tipo admite 80, 68 y 55 pasajeros. 
Quiere encontrar la mejor combinación de estos tipos para llevar a 372 pasajeros para tener el minimo coste . 
Tenemos una tabla del coste de cada tipo de avion, cuanto cuesta usar de 1-5 aviones. 
Además, tenemos un coste fijo de 6k € para cada tipo de avión que se usa. 





#### Creamos el modelo 

In [2]:
modelo = pe.ConcreteModel()


#### Sets 

${i}$: tipo de avión

${j}$: numero de aviones 



In [3]:
modelo.aviones= pe.Set(initialize=['A', 'B', 'C'])
modelo.numero_aviones = pe.Set(initialize=[1, 2, 3, 4, 5])


#### Parámetros 
$Q_{i}$: capacidad del avión i (número de pasajeros máximo)



In [4]:
modelo.capacidad= pe.Param(modelo.aviones, initialize={'A': 80, 'B':68, 'C': 55})

$FC$: costes fijos [en miles de €]



In [5]:
modelo.coste_fijo = pe.Param(initialize=6)

$C_{ij}$ : coste de usar j aviones del tipo i

In [6]:
costes = {('A', 1): 11, ('A', 2): 20, ('A', 3): 30, ('A', 4): 40, ('A', 5): 50,
            ('B', 1): 9, ('B', 2): 17, ('B', 3): 24, ('B', 4): 34, ('B', 5): 45,
            ('C', 1): 8, ('C', 2): 15, ('C', 3): 21, ('C', 4): 26, ('C', 5): 31}

modelo.costes = pe.Param(modelo.aviones, modelo.numero_aviones, initialize=costes)

#### Variables

$x_{ij}$ : se usan j aviones del tipo i [bool]. 

In [7]:
modelo.x = pe.Var(modelo.aviones, modelo.numero_aviones, domain=pe.Binary)

#### Función objetivo 

Queremos minimizar los costes. 
Para cada tipo de avión (i) y cada numero de avion j, si se elije esa combinación (xij=1), 
entonces se le suman los costes. los costes son los fijos (FC) + los variables (el coste de esa celda en la matriz de costes C)


min $\sum_{ij} (C_{ij} + FC) * x_{ij} $

In [8]:
def funcion_objetivo(modelo):
    fc = modelo.coste_fijo

    return sum((modelo.costes[i,j] + fc) * modelo.x[i,j] for i in modelo.aviones for j in modelo.numero_aviones)

modelo.coste = pe.Objective(rule=funcion_objetivo, sense=pe.minimize)


#### Constrains 

Restricción 1) El número de pasajeros debe ser igual o superior a 372 



$\sum_{i,j}j·Q_{i}·x_{i,j} \ge 372$


In [12]:
modelo.passenger_number_constraint=pe.ConstraintList()

modelo.passenger_number_constraint.add(
sum(modelo.capacidad[i]*modelo.x[i,j]*j for i in modelo.aviones for j in modelo.numero_aviones) >=372
)      

passenger_number_constraint (type=<class
'pyomo.core.base.constraint.ConstraintList'>) on block unknown with a new
Component (type=<class 'pyomo.core.base.constraint.ConstraintList'>). This is
usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().


Restricción 2) Para cada avión i, solo puede haber una xij que tenga un 1. 

$\sum_j x_{ij}\le 1$

In [ ]:
modelo.airplane_number_constraint=pe.ConstraintList()

for i in modelo.aviones:
    modelo.airplane_number_constraint.add(
    sum(modelo.x[i,j] for j in modelo.numero_aviones) <=1
)     

In [15]:
#solver = po.SolverFactory('glpk')
solver = po.SolverFactory('gurobi')
results = solver.solve (modelo, tee=True)

Set parameter Username
Set parameter LicenseID to value 2703329
Academic license - for non-commercial use only - expires 2026-09-04
Read LP format model from file C:\Users\macar\AppData\Local\Temp\tmpc46hwef1.pyomo.lp
Reading time = 0.02 seconds
x1: 4 rows, 15 columns, 30 nonzeros
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i5-10210U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 15 columns and 30 nonzeros
Model fingerprint: 0x50666697
Variable types: 0 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [1e+01, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Found heuristic solution: objective 56.0000000
Presolve time: 0.02s
Presolved: 4 rows, 15 columns, 30 nonzeros
Variable types: 0 continuous, 15 integer (15 binary)

Root re

In [16]:

print(pe.value(modelo.coste))

56.0


In [20]:
for i in modelo.aviones:
    for j in modelo.numero_aviones:
        if pe.value(modelo.x[i, j]) == 1:
            print('Se han elegido {} aviones de tipo {}'.format(j, i))

Se han elegido 5 aviones de tipo A


In [21]:
# Capacidad total transportada
total_pasajeros = sum(
    pe.value(modelo.x[i, j]) * modelo.capacidad[i] * j
    for i in modelo.aviones
    for j in modelo.numero_aviones
)
print('Se transportan un total de', total_pasajeros, 'pasajeros')

Se transportan un total de 400.0 pasajeros


In [24]:
modelo.pprint()

2 Set Declarations
    aviones : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'A', 'B', 'C'}
    numero_aviones : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}

3 Param Declarations
    capacidad : Size=3, Index=aviones, Domain=Any, Default=None, Mutable=False
        Key : Value
          A :    80
          B :    68
          C :    55
    coste_fijo : Size=1, Index=None, Domain=Any, Default=None, Mutable=False
        Key  : Value
        None :     6
    costes : Size=15, Index=aviones*numero_aviones, Domain=Any, Default=None, Mutable=False
        Key      : Value
        ('A', 1) :    11
        ('A', 2) :    20
        ('A', 3) :    30
        ('A', 4) :    40
        ('A', 5) :    50
        ('B', 1) :     9
        ('B', 2) :    17
        ('B', 3) :    24
        ('B', 4) :    34
        ('B', 5) :    45

In [25]:
modelo.display() 

Model unknown

  Variables:
    x : Size=15, Index=aviones*numero_aviones
        Key      : Lower : Value : Upper : Fixed : Stale : Domain
        ('A', 1) :     0 :   0.0 :     1 : False : False : Binary
        ('A', 2) :     0 :   0.0 :     1 : False : False : Binary
        ('A', 3) :     0 :   0.0 :     1 : False : False : Binary
        ('A', 4) :     0 :   0.0 :     1 : False : False : Binary
        ('A', 5) :     0 :   1.0 :     1 : False : False : Binary
        ('B', 1) :     0 :   0.0 :     1 : False : False : Binary
        ('B', 2) :     0 :   0.0 :     1 : False : False : Binary
        ('B', 3) :     0 :   0.0 :     1 : False : False : Binary
        ('B', 4) :     0 :   0.0 :     1 : False : False : Binary
        ('B', 5) :     0 :   0.0 :     1 : False : False : Binary
        ('C', 1) :     0 :   0.0 :     1 : False : False : Binary
        ('C', 2) :     0 :   0.0 :     1 : False : False : Binary
        ('C', 3) :     0 :   0.0 :     1 : False : False : Binary
  